In [1]:
import sys
sys.path.insert(0, '..')

In [9]:
import torch
import tensor

ModuleNotFoundError: No module named 'tensor'

In [3]:
# Read the file and tokenize each line into words
file_path = "D:\XGNN\HAN-master\data\DBLP_four_area\\author_label.txt"  # Replace with the path to your file
bag_of_words = []

# Vocabulary to store unique words
vocabulary = set()

ids = []
labels = []

with open(file_path, "r") as file:
    for idx, line in enumerate(file):
        line = line.strip()
        id, label, name = line.split("\t")
        ids.append(id)
        labels.append(int(label))

sorted_lists = sorted(zip(ids, labels))

author_id_dict = {}
for idx, id in enumerate(ids):
    author_id_dict[id] = idx
# Unzip the sorted lists
ids, labels = zip(*sorted_lists)


In [4]:
from collections import defaultdict
import re

# Read the file and tokenize each line into words
file_path = "D:\XGNN\HAN-master\data\DBLP_four_area\paper.txt"  # Replace with the path to your file
bag_of_words = []

# Vocabulary to store unique words
vocabulary = []
ids = []
total_words = {}
paper_id_dict = {}

with open(file_path, "r") as file:
    for idx, line in enumerate(file):
        line = line.strip()
        id, text = line.split("\t", 1)
        words = re.findall(r'\w+', text.lower())  # Tokenize and convert to lowercase
        word_count = defaultdict(int)
        for word in words:
            #if word in stop_words:
            #    continue
            if word not in vocabulary:
                vocabulary.append(word)
            if word in total_words:
                total_words[word] += 1
            else:
                total_words[word] = 1
            word_count[word] += 1
        bag_of_words.append(dict(word_count))
        paper_id_dict[id] = idx

print(total_words)

{'some': 48, 'philosophical': 3, 'problems': 117, 'with': 1041, 'formal': 38, 'learning': 937, 'theory': 133, 'terminological': 13, 'cycles': 5, 'in': 2842, 'kl': 2, 'one': 56, 'based': 1197, 'knowledge': 446, 'representation': 126, 'languages': 64, 'integrating': 86, 'description': 59, 'logics': 45, 'and': 2921, 'action': 62, 'formalisms': 4, 'first': 65, 'results': 115, 'old': 3, 'resolution': 52, 'meets': 9, 'modern': 10, 'sls': 1, 'partial': 69, 'compilation': 14, 'of': 3639, 'strategic': 7, 'using': 1045, 'an': 968, 'annotated': 5, 'language': 346, 'corpus': 16, 'as': 132, 'a': 3341, 'virtual': 26, 'stochastic': 48, 'grammar': 13, 'plan': 43, 'abstraction': 34, 'on': 1125, 'operator': 24, 'generalization': 37, 'quantifying': 6, 'incentive': 2, 'compatibility': 2, 'ranking': 123, 'systems': 552, 'case': 130, 'introspection': 4, 'quonto': 1, 'querying': 125, 'ontologies': 22, 'editorial': 1, 'comprehension': 8, 'oped': 1, 'through': 153, 'argument': 10, 'units': 6, 'looking': 4, 'fo

In [12]:
total_words = dict(sorted(total_words.items(), key=lambda item: item[1], reverse=True))

In [6]:
file_path = "D:\XGNN\HAN-master\data\DBLP_four_area\paper_author.txt"  # Replace with the path to your file

mappings = []

with open(file_path, "r") as file:
    for idx, line in enumerate(file):
        line = line.strip()
        paper,author = line.split("\t", 1)
        if paper in paper_id_dict and author in author_id_dict:
            mappings.append([paper_id_dict[paper],author_id_dict[author]])

mappings = torch.tensor(mappings)
print(mappings)

tensor([[    0,   438],
        [    1,  1945],
        [    2,  1945],
        ...,
        [14375,   470],
        [14375,  1549],
        [14375,   334]])


In [13]:
total_words

{'for': 4390,
 'of': 3639,
 'a': 3341,
 'and': 2921,
 'in': 2842,
 'the': 2132,
 'data': 1856,
 'based': 1197,
 'to': 1164,
 'on': 1125,
 'using': 1045,
 'with': 1041,
 'an': 968,
 'learning': 937,
 'mining': 785,
 'database': 726,
 'information': 724,
 'query': 721,
 'system': 687,
 'retrieval': 658,
 'web': 643,
 'databases': 617,
 'search': 594,
 'from': 589,
 'efficient': 573,
 'systems': 552,
 'queries': 521,
 'approach': 509,
 'model': 500,
 'clustering': 458,
 'knowledge': 446,
 'analysis': 393,
 'text': 381,
 'by': 378,
 'classification': 375,
 'distributed': 367,
 'xml': 362,
 'time': 357,
 'large': 354,
 'language': 346,
 'multi': 344,
 'models': 344,
 'processing': 344,
 'algorithm': 319,
 'relational': 317,
 'object': 308,
 'management': 298,
 'document': 265,
 'semantic': 257,
 'algorithms': 257,
 'evaluation': 251,
 'networks': 243,
 'rules': 237,
 'dynamic': 236,
 'oriented': 234,
 'performance': 232,
 'design': 230,
 'selection': 229,
 'reasoning': 224,
 'framework': 22

In [ ]:
matrix = []
for item in bag_of_words:
    vector = [item.get(word, 0) for word in vocabulary]
    matrix.append(vector)

# Convert the matrix to a PyTorch tensor
tensor = torch.tensor(matrix, dtype=torch.float32)
print(tensor)


In [7]:
from torch_geometric.data import HeteroData


dataset = HeteroData()
dataset['author'].x = torch.tensor([0] * len(labels)).unsqueeze(1)
dataset['author'].y = torch.tensor(list(labels))
dataset['paper'].x = tensor
#data['conference'].x = torch.tensor([[0],[1]])
##data['author', 'writes', 'paper'].edge_index = torch.tensor([[0,2,0,0,2,2],[0,1,2,3,0,3]])
dataset['author', 'writes', 'paper'].edge_index = mappings.t()
#data['paper', 'published', 'conference'].edge_index = torch.tensor([[0,2],[0,1]])
dataset

NameError: name 'tensor' is not defined

In [ ]:
from XGNN.Infrastructure.DataLoader import DataLoader
data_loader = DataLoader(root="./datasets/dblp")
dataset2 = data_loader._load_dblp(1)
dataset2

In [ ]:
from XGNN.Infrastructure.Evaluator import Evaluator
evaluator = Evaluator(dataset)
hgnn = HGNN(d=dataset2)

In [ ]:
for _ in range(20):
    print(_, hgnn.train_model())

In [ ]:
pred = hgnn.predict_all(new_data=dataset)

gnn_results = set()
for i in pred.keys():
    for ii in range(len(pred[i])):
        if pred[i][ii] == label_to_explain:
            gnn_results.add((ii, i))

In [ ]:
from XGNN.Infrastructure.BaseExplainer import BaseExplainer
base_explainer = BaseExplainer(gnn=hgnn, data=dataset)
base_result = base_explainer.explain(label_to_explain)
base_result

In [ ]:
from ontolearn.owlapy.render import DLSyntaxObjectRenderer
renderer = DLSyntaxObjectRenderer()
print(renderer.render(base_result))

In [ ]:
dataset

In [ ]:
explainer = DiscriminativeExplainer(hgnn, dataset, "http://example.org/", owl_graph_path = "./owlGraphs/example4.owl", generate_new_owl_file=True)

In [ ]:
from ontolearn.metrics import Accuracy, F1
hypotheses, model = explainer.explain(1, 5, debug=True, max_runtime=60, num_generations=400, quality_func=F1())

In [ ]:
[print(h) for h in hypotheses]

In [ ]:
discriminative_result = hypotheses[0].concept
print(renderer.render(discriminative_result))

In [ ]:
from rich.table import Table
table = Table("Approach", "Precision", "Recall", "Accuracy", "F1 Score", "Explanation", show_lines=True)

In [ ]:
base_score = evaluator.explanation_accuracy(gnn_results, base_result)
base_f1 = evaluator.f1_score(gnn_results, base_result)
table.add_row("Base Explanation",
                f'{base_score[0] * 100:.2f}%',
                f'{base_score[1] * 100:.2f}%',
                f'{base_score[2] * 100:.2f}%',
                f'{base_f1:.4f}',
                renderer.render(base_result))

In [ ]:
for discriminative_result in hypotheses:
    discriminative_score = evaluator.explanation_accuracy(gnn_results, discriminative_result.concept)
    discriminative_f1 = evaluator.f1_score(gnn_results, discriminative_result.concept)
    table.add_row("Discriminative Explanation",
                    f'{discriminative_score[0] * 100:.2f}%',
                    f'{discriminative_score[1] * 100:.2f}%',
                    f'{discriminative_score[2] * 100:.2f}%',
                    f'{discriminative_f1:.4f}',
                    renderer.render(discriminative_result.concept))

In [ ]:
from rich.console import Console
console = Console()
console.print(table)

In [ ]:
discriminative_score

In [ ]:
#import torch
#from collections import defaultdict
#import re

## Read the file and tokenize each line into words
#file_path = "D:\XGNN\HAN-master\data\DBLP_four_area\paper.txt"  # Replace with the path to your file
#bag_of_words = []

## Vocabulary to store unique words
#vocabulary = set()

#ids = []
#word_counts = []

#with open(file_path, "r") as file:
#    for idx, line in enumerate(file):
#        if idx >= 5:
#            break
#        line = line.strip()
#        id, text = line.split("\t", 1)
#        words = re.findall(r'\w+', text.lower())  # Tokenize and convert to lowercase
#        word_count = {}
#        for word in words:
#            vocabulary.add(word)  # Add word to vocabulary
#            if word in word_count:
#                word_count[word] += 1
#            else:
#                word_count[word] = 1
#        ids.append(id)
#        word_counts.append(word_count)

## Create a tensor for each document
#tensors = []
#sorted_lists = sorted(zip(ids, word_counts))
#ids, word_counts = zip(*sorted_lists)

#print(bag_of_words)

#for item in bag_of_words:
#    vector = [item["word_count"].get(word, 0) for word in vocabulary]
#    tensor = torch.tensor(vector, dtype=torch.float32)
#    print(tensor)
#    tensors.append(tensor)

#id_dict = {}
#for idx, id in enumerate(ids):
#    id_dict[id] = idx

In [ ]:
print(vocabulary)

In [ ]:
explainer = DiscriminativeExplainer(hgnn, data, "http://example.org/", owl_graph_path = "./owlGraphs/example4.owl", generate_new_owl_file=True)